### Pacotes necessários

In [33]:
import pandas as pd
import sys
import scipy
import matplotlib 
import seaborn
import numba
import gmpy2
import pysubgroup as ps
import numpy as np
sys.path.append('./SSDpp-numeric-master')
sys.path
from src.util.results2folder import attach_results,print2folder
from _classes import SSDC

### Leitura do banco de dados

In [34]:
df = pd.read_csv('./data/dengue_internacao.csv', sep=',')

/tmp/ipykernel_2478848/3527530056.py:1: DtypeWarning: Columns (4,16,21,107,113,119,142,150) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/dengue_internacao.csv', sep=',')


In [35]:
# selecionando as colunas desejadas
df_selecionado = df[['ano', 'data_notificacao', 'sigla_uf_notificacao', 'data_primeiros_sintomas', 'idade_paciente', 
                     'sexo_paciente', 'raca_cor_paciente', 'escolaridade_paciente', 'gestante_paciente', 
                     'possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias',
                     'data_internacao','classificacao_final','evolucao_caso', 'data_obito', 'data_encerramento']]

df_selecionado = df_selecionado[df_selecionado['ano'] >= 2023]


### Tratamento do banco de dados

In [36]:
# transformando atributo nominal data_obito para binário houve_obito
df_selecionado['houve_obito'] = df_selecionado['data_obito'].apply(lambda x: 0 if pd.isna(x) else 1)

# remoção data_obito
df_selecionado = df_selecionado.drop(columns=['data_obito'])

In [37]:
# removendo as linhas onde não houveram datas de primeiros sintomas ou data da notificação.
df_selecionado = df_selecionado.dropna(subset=['data_primeiros_sintomas', 'data_notificacao'])

df_selecionado['data_notificacao'] = pd.to_datetime(df_selecionado['data_notificacao'], errors='coerce')
df_selecionado['data_primeiros_sintomas'] = pd.to_datetime(df_selecionado['data_primeiros_sintomas'], errors='coerce')
# criando a coluna 'tempo_internacao'
df_selecionado['intervalo_busca_atendimento'] = df_selecionado.apply(
    lambda row: (row['data_notificacao'] - row['data_primeiros_sintomas']).days if pd.notna(row['data_primeiros_sintomas']) else 0,
    axis=1
)

In [38]:
# removendo as linhas onde não houveram internações ou data de encerramento.
df_selecionado = df_selecionado.dropna(subset=['data_internacao', 'data_encerramento'])

# convertendo as colunas para datetime
df_selecionado['data_encerramento'] = pd.to_datetime(df_selecionado['data_encerramento'], errors='coerce')
df_selecionado['data_internacao'] = pd.to_datetime(df_selecionado['data_internacao'], errors='coerce')

# criando a coluna 'intervalo_sintoma_internacao' que é a data dos primeiros sintomas até a data que ela foi internada.
df_selecionado['intervalo_sintoma_internacao'] = df_selecionado.apply(
    lambda row: (row['data_internacao'] - row['data_primeiros_sintomas']).days if pd.notna(row['data_internacao']) else 0,
    axis=1
)

# criando a coluna 'intervalo_internacao'
df_selecionado['intervalo_internacao'] = df_selecionado.apply(
    lambda row: (row['data_encerramento'] - row['data_internacao']).days if pd.notna(row['data_internacao']) else 0,
    axis=1
)

df_selecionado = df_selecionado.query("intervalo_internacao >= 0 and intervalo_internacao <= 90 and" + " intervalo_sintoma_internacao >= 0 and intervalo_sintoma_internacao <= 90 and"
              + " intervalo_busca_atendimento >= 0 and intervalo_busca_atendimento <= 90")

In [39]:
# prompt: na variável, idade_paciente, retirar manter os valores depois de "-" e transformar em inteiro
df_selecionado['idade_paciente'] = df_selecionado['idade_paciente'].str.split('-').str[1]
df_selecionado['idade_paciente'] = df_selecionado['idade_paciente'].str.extract('(\d+)').astype(int)

In [40]:
# drop das variáveis data_notificacao, data_primeiros_sintomas, data_internacao e data_encerramento
df_selecionado = df_selecionado.drop(['data_notificacao', 'data_primeiros_sintomas', 'data_internacao', 'data_encerramento'], axis=1)

In [41]:
# recodificando as variáveis categóricas
raca_cor_map = {
    1:"Branca",
    2:"Preta",
    3:"Amarela",
    4:"Parda",
    5:"Indígena",
    9:"Ignorado"
}
df_selecionado["raca_cor_paciente"] = df_selecionado["raca_cor_paciente"].map(raca_cor_map)

escolaridade_map = {
    0: "Analfabeto",
    1: "1ª a 4ª série incompleta do EF",
    2: "4ª série completa do EF",
    3: "5ª A 8ª série incomplet do EF",
    4: "Ensino fundamental incompleto",
    5: "Ensino médio incompleto",
    6: "Ensino médio completo",
    7: "Educação superior incompleta",
    8: "Educação superior completa",
    9: "Ignorado",
    10: "Não se Aplica"
}

df_selecionado["escolaridade_paciente"] = df_selecionado["escolaridade_paciente"].map(escolaridade_map)

gestante_map = {
    1:"1ºTrimestre", 2:"2ºTrimestre", 3:"3ºTrimestre", 4:"Idade gestacional Ignorada", 5:"Não", 6:"Não se aplica",9:"Ignorado"
}

df_selecionado["gestante_paciente"] = df_selecionado["gestante_paciente"].map(gestante_map)

dicotomia_map = {
    1: "Sim",
    2: "Não"
}

col_dicotomia = ['possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias']

for col in col_dicotomia:
    df_selecionado[col] = df_selecionado[col].map(dicotomia_map)

classificacao_map={
    5: "Descartado",
    10: "Dengue", 
    11: "Dengue com Sinais de Alarme",
    12: "Dengue Grave", 
    13: "Chikungunya"
}

df_selecionado['classificacao_final'] = df_selecionado['classificacao_final'].map(classificacao_map).fillna(np.nan)

evolucao_map={
    1:"Cura",
    2:"Óbito pelo agravo", 
    3:"Óbito por outras causas",
    4:"Óbito em investigação", 
    9:"Ignorado",
}

df_selecionado['evolucao_caso']=df_selecionado['evolucao_caso'].map(evolucao_map).fillna(np.nan)
df_selecionado['houve_obito'] = df_selecionado['houve_obito'].map({1: 'sim', 0: 'não'})

df_selecionado

,ano,sigla_uf_notificacao,idade_paciente,sexo_paciente,raca_cor_paciente,escolaridade_paciente,gestante_paciente,possui_doenca_autoimune,possui_diabetes,possui_doencas_hematologicas,...,apresenta_artralgia,apresenta_artrite,apresenta_leucopenia,apresenta_petequias,classificacao_final,evolucao_caso,houve_obito,intervalo_busca_atendimento,intervalo_sintoma_internacao,intervalo_internacao
118790,2024,AM,55,F,Parda,5ª A 8ª série incomplet do EF,Não,Não,Não,Não,...,Sim,Não,Sim,Não,NaN,NaN,não,6,6,61
118792,2024,AM,88,M,Parda,NaN,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue Grave,Óbito pelo agravo,sim,4,3,61
118793,2024,AM,54,M,Parda,Educação superior completa,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,4,1,3
118794,2024,AM,48,F,Parda,Ensino médio completo,Não,Não,Não,Não,...,Não,Não,Não,Sim,Dengue Grave,Cura,não,4,1,22
118795,2024,AM,83,F,Parda,Ensino médio incompleto,Não,Não,Não,Não,...,Não,Não,Não,Não,Dengue com Sinais de Alarme,Cura,não,1,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257951,2023,SP,54,M,Branca,Ensino médio completo,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,1,0,9
257952,2023,SP,39,M,Branca,NaN,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,4,4,1
257953,2023,SP,13,F,Branca,Ensino médio incompleto,Não se aplica,Não,Não,Não,...,Não,Não,Sim,Não,Dengue com Sinais de Alarme,Cura,não,3,4,25
257954,2023,SP,75,M,Branca,Ignorado,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,31,30,2


In [42]:
## retirando os resultados NaN do banco de dados
df_selecionado = df_selecionado.dropna()


In [30]:
# transformar variáveis nominais em categóricas
df_aux = df_selecionado
categoricas_var = ['sigla_uf_notificacao','sexo_paciente','raca_cor_paciente','escolaridade_paciente','gestante_paciente','possui_doenca_autoimune', 'possui_diabetes', 'possui_doencas_hematologicas', 
                     'possui_hepatopatias', 'possui_doenca_renal', 'possui_hipertensao', 
                     'possui_doenca_acido_peptica', 'apresenta_febre','apresenta_cefaleia','apresenta_exantema','apresenta_dor_costas','apresenta_mialgia','apresenta_vomito','apresenta_conjutivite','apresenta_dor_retroorbital',
                     'apresenta_artralgia','apresenta_artrite','apresenta_leucopenia','apresenta_petequias','classificacao_final','evolucao_caso','houve_obito']

for col in categoricas_var:
    df_selecionado[col]=pd.Categorical(df_selecionado[col])

df_selecionado

/tmp/ipykernel_2478848/3418255390.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado[col]=pd.Categorical(df_selecionado[col])
/tmp/ipykernel_2478848/3418255390.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado[col]=pd.Categorical(df_selecionado[col])
/tmp/ipykernel_2478848/3418255390.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,ano,sigla_uf_notificacao,idade_paciente,sexo_paciente,raca_cor_paciente,escolaridade_paciente,gestante_paciente,possui_doenca_autoimune,possui_diabetes,possui_doencas_hematologicas,...,apresenta_artralgia,apresenta_artrite,apresenta_leucopenia,apresenta_petequias,classificacao_final,evolucao_caso,houve_obito,intervalo_busca_atendimento,intervalo_sintoma_internacao,intervalo_internacao
118793,2024,AM,54,M,Parda,Educação superior completa,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,4,1,3
118794,2024,AM,48,F,Parda,Ensino médio completo,Não,Não,Não,Não,...,Não,Não,Não,Sim,Dengue Grave,Cura,não,4,1,22
118795,2024,AM,83,F,Parda,Ensino médio incompleto,Não,Não,Não,Não,...,Não,Não,Não,Não,Dengue com Sinais de Alarme,Cura,não,1,0,9
118808,2024,AC,42,F,Parda,Ensino médio completo,Não,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,2,0,19
118810,2024,AC,52,F,Parda,Ignorado,Não se aplica,Não,Não,Não,...,Sim,Sim,Sim,Sim,Dengue,Cura,não,4,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257950,2023,SP,45,F,Branca,Educação superior completa,Não,Não,Não,Não,...,Não,Não,Não,Não,Dengue Grave,Óbito pelo agravo,sim,1,6,3
257951,2023,SP,54,M,Branca,Ensino médio completo,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,1,0,9
257953,2023,SP,13,F,Branca,Ensino médio incompleto,Não se aplica,Não,Não,Não,...,Não,Não,Sim,Não,Dengue com Sinais de Alarme,Cura,não,3,4,25
257954,2023,SP,75,M,Branca,Ignorado,Não se aplica,Não,Não,Não,...,Não,Não,Não,Não,Dengue,Cura,não,31,30,2


### Teste intervalo_internacao BeamSearch

In [100]:
# Carregar banco df_selecionado

data = df_selecionado


#target = ps.BinaryTarget ('sobrevivencia', True)
target2 = ps.NumericTarget('intervalo_internacao')
searchspace = ps.create_selectors(data, ignore=['intervalo_internacao'])
task = ps.SubgroupDiscoveryTask (
    data,
    target2,
    searchspace,
    result_set_size=300,
    depth=3,
    qf=ps.StandardQFNumeric(1.0))
result = ps.BeamSearch(beam_width=350).execute(task)
print(result.to_dataframe())

          quality                                           subgroup  size_sg  \
0    87632.420816  intervalo_sintoma_internacao<1 AND possui_doen...     3576   
1    87577.379148                     intervalo_sintoma_internacao<1     3595   
2    87165.510734  intervalo_sintoma_internacao<1 AND possui_doen...     3487   
3    87146.190555  intervalo_sintoma_internacao<1 AND possui_doen...     3537   
4    87081.587488  intervalo_sintoma_internacao<1 AND possui_doen...     3500   
..            ...                                                ...      ...   
295  46481.076603  apresenta_vomito=='Não' AND intervalo_sintoma_...     2293   
296  46385.813430  apresenta_leucopenia=='Não' AND intervalo_sint...     2509   
297  46227.688517  apresenta_dor_costas=='Não' AND escolaridade_p...     1198   
298  45908.782753  apresenta_petequias=='Não' AND apresenta_vomit...     2139   
299  45869.142365  apresenta_artrite=='Não' AND intervalo_sintoma...     2695   

     size_dataset    mean_s

In [108]:
result_beam_search = result.to_dataframe()
#Calculando o WRAcc do Beam Search
result_beam_search['WRAcc'] = (result_beam_search['size_sg']/result_beam_search['size_dataset'])*(result_beam_search['mean_sg']-result_beam_search['mean_dataset'])

### Teste intervalo_internacao DFS

In [102]:
# Carregar banco df_selecionado

data = df_selecionado


#target = ps.BinaryTarget ('sobrevivencia', True)
target2 = ps.NumericTarget('intervalo_internacao')
searchspace = ps.create_selectors(data, ignore=['intervalo_internacao'])
task = ps.SubgroupDiscoveryTask (
    data,
    target2,
    searchspace,
    result_set_size=300,
    depth=2,
    qf=ps.StandardQFNumeric(1.0))
result = ps.DFS().execute(task)
print(result.to_dataframe())

          quality                                           subgroup  size_sg  \
0    87632.420816  intervalo_sintoma_internacao<1 AND possui_doen...     3576   
1    87577.379148                     intervalo_sintoma_internacao<1     3595   
2    87081.587488  intervalo_sintoma_internacao<1 AND possui_doen...     3500   
3    87062.267309  intervalo_sintoma_internacao<1 AND possui_doen...     3550   
4    86608.699327  intervalo_sintoma_internacao<1 AND possui_hepa...     3545   
..            ...                                                ...      ...   
295  12409.863975  houve_obito=='sim' AND possui_doenca_renal=='Não'      902   
296  12396.166158  apresenta_vomito=='Não' AND escolaridade_pacie...     7676   
297  12365.479845  apresenta_leucopenia=='Sim' AND possui_doenca_...     6309   
298  12357.391407                   ano==2023 AND sexo_paciente=='F'    17173   
299  12298.124943  apresenta_petequias=='Não' AND houve_obito=='sim'      855   

     size_dataset    mean_s

In [103]:
result_dfs=result.to_dataframe()
#Calculando o WRAcc do DFS
result_dfs['WRAcc'] = (result_dfs['size_sg']/result_dfs['size_dataset'])*(result_dfs['mean_sg']-result_dfs['mean_dataset'])

### 	Teste intervalo_internação SSDC

In [31]:
df_selecionado.dtypes

ano                                int64
sigla_uf_notificacao            category
idade_paciente                     int32
sexo_paciente                   category
raca_cor_paciente               category
escolaridade_paciente           category
gestante_paciente               category
possui_doenca_autoimune         category
possui_diabetes                 category
possui_doencas_hematologicas    category
possui_hepatopatias             category
possui_doenca_renal             category
possui_hipertensao              category
possui_doenca_acido_peptica     category
apresenta_febre                 category
apresenta_cefaleia              category
apresenta_exantema              category
apresenta_dor_costas            category
apresenta_mialgia               category
apresenta_vomito                category
apresenta_conjutivite           category
apresenta_dor_retroorbital      category
apresenta_artralgia             category
apresenta_artrite               category
apresenta_leucop

In [32]:
task_name = "discovery"
target_type = "numeric"

# load class and fit to data
model = SSDC(task = task_name,max_depth=3, beam_width=25,max_rules=20,n_cutpoints=3)
model.fit(df_selecionado)

d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4 4 4 ... 1 1 1]' ha

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

In [36]:
print(model)
pd.DataFrame(model.antecedent_description)

IF x in raca_cor_paciente = Ignorado AND possui_hipertensao = Sim AND 16 <= idade_paciente <= 35 THEN mean = 1154.608695652174; std = 5295.815727109558 ,  
ELSE IF x in sigla_uf_notificacao = SC AND intervalo_busca_atendimento <= 1 AND apresenta_artrite = Sim THEN mean = 8.49090909090909; std = 3.7467341150561535 ,  
ELSE IF x in sigla_uf_notificacao = SC AND apresenta_conjutivite = Sim AND apresenta_vomito = Não THEN mean = 9.530973451327434; std = 6.952846788999286 ,  
ELSE IF x in sigla_uf_notificacao = SC AND apresenta_conjutivite = Sim AND escolaridade_paciente = 5ª A 8ª série incomplet do EF THEN mean = 9.516666666666667; std = 7.541348980417527 ,  
ELSE IF x in sigla_uf_notificacao = SC AND escolaridade_paciente = 4ª série completa do EF AND intervalo_busca_atendimento <= 6 THEN mean = 10.53225806451613; std = 7.316414200507096 ,  
ELSE IF x in escolaridade_paciente = 5ª A 8ª série incomplet do EF AND sigla_uf_notificacao = RJ AND intervalo_busca_atendimento <= 6 THEN mean = 11.

""


In [49]:
model2 = SSDC(task = task_name, max_depth=4, beam_width=25,max_rules=20,n_cutpoints=3)
model2.fit(df_selecionado)

d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4 4 4 ... 1 1 1]' ha

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

In [50]:
model3 = SSDC(task = task_name,max_depth=3, beam_width=25,max_rules=20,n_cutpoints=5)
model3.fit(df_selecionado)

d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4 4 4 ... 1 1 1]' ha

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

In [51]:
model4 = SSDC(task = task_name,max_depth=4, beam_width=25,max_rules=20,n_cutpoints=5)
model4.fit(df_selecionado)

d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4 4 4 ... 1 1 1]' ha

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

In [52]:
model5 = SSDC(task = task_name,max_depth=3, beam_width=50,max_rules=20,n_cutpoints=3)
model5.fit(df_selecionado)

d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4 4 4 ... 1 1 1]' ha

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

In [53]:
model6 = SSDC(task = task_name,max_depth=4, beam_width=50,max_rules=20,n_cutpoints=3)
model6.fit(df_selecionado)

d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 2  2  2 ... 24 24 24]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 0 0 ... 0 2 0]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  df.loc[:,colname] = df.loc[:,colname].cat.codes #transform this column to codes
d:\comum\disciplina_isolada\SSDpp-numeric-master\src\util\_read_dataset.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4 4 4 ... 1 1 1]' ha

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
Iteration: 72
I

### Estatísticas de Qualidade por modelo no SSDC

#### Modelo 1

In [64]:
model_estat = pd.DataFrame(model.statistic_rules)

In [72]:
df_model = pd.DataFrame(model.antecedent_description, columns=['Subgrupros'])

In [73]:
result_model = pd.concat([df_model, model_estat], axis=1)

In [76]:
result_model['WRAcc'] = (result_model['usage']/result_model['usage_default'])*((result_model['mean'])-(result_model['RSS_default_pattern']/result_model['usage_default']))

In [77]:
result_model

,Subgrupros,usage,mean,mean2,variance2,RSS2,variance,RSS_default_pattern,usage_default,RSS_default_uncovered,WRAcc
0,raca_cor_paciente = Ignorado AND possui_hipert...,23,1154.608696,26.0,25.00,71.959843,2.804566e+07,6.745869e+08,33096,1.893931e+08,-1.336255e+01
1,sigla_uf_notificacao = SC AND intervalo_busca_...,55,8.490909,17.5,0.25,54.297564,1.403802e+01,1.131583e+04,33041,1.893814e+08,1.356387e-02
2,sigla_uf_notificacao = SC AND apresenta_conjut...,113,9.530973,20.0,16.00,46.433528,4.834208e+01,2.438758e+04,32928,1.893566e+08,3.016607e-02
3,sigla_uf_notificacao = SC AND apresenta_conjut...,60,9.516667,19.0,25.00,77.179143,5.687194e+01,1.329205e+04,32868,1.893430e+08,1.663428e-02
4,sigla_uf_notificacao = SC AND escolaridade_pac...,62,10.532258,24.0,4.00,11.451071,5.352992e+01,1.225892e+04,32806,1.893306e+08,1.919868e-02
...,...,...,...,...,...,...,...,...,...,...,...
276,apresenta_vomito = Não AND apresenta_exantema ...,36,29.527778,23.5,0.25,1.823878,5.772708e+02,2.227515e+04,490,1.034097e+07,-1.170493e+00
277,apresenta_dor_costas = Não AND 2 <= intervalo_...,19,29.263158,20.5,20.25,50.060721,3.759363e+02,4.663079e+03,471,1.026562e+07,7.810887e-01
278,possui_hipertensao = Não AND classificacao_fin...,290,65.448276,22.5,0.25,0.569493,1.140199e+04,3.839002e+06,181,6.426248e+06,-3.387794e+04
279,apresenta_artrite = Não,165,109.709091,23.5,0.25,1.823878,2.985923e+04,6.137324e+06,16,2.456113e+05,-3.954566e+06


#### Modelo 2

In [78]:
model_estat2 = pd.DataFrame(model2.statistic_rules)
df_model2 = pd.DataFrame(model2.antecedent_description, columns=['Subgrupros'])
result_model2 = pd.concat([df_model2, model_estat2], axis=1)
result_model2['WRAcc'] = (result_model2['usage']/result_model2['usage_default'])*((result_model2['mean'])-(result_model2['RSS_default_pattern']/result_model2['usage_default']))

#### Modelo 3

In [80]:
model_estat3 = pd.DataFrame(model3.statistic_rules)
df_model3 = pd.DataFrame(model3.antecedent_description, columns=['Subgrupros'])
result_model3 = pd.concat([df_model3, model_estat3], axis=1)
result_model3['WRAcc'] = (result_model3['usage']/result_model3['usage_default'])*((result_model3['mean'])-(result_model3['RSS_default_pattern']/result_model3['usage_default']))

#### Modelo 4

In [81]:
model_estat4 = pd.DataFrame(model4.statistic_rules)
df_model4 = pd.DataFrame(model4.antecedent_description, columns=['Subgrupros'])
result_model4 = pd.concat([df_model4, model_estat4], axis=1)
result_model4['WRAcc'] = (result_model4['usage']/result_model4['usage_default'])*((result_model4['mean'])-(result_model4['RSS_default_pattern']/result_model4['usage_default']))

#### Modelo 5

In [82]:
model_estat5 = pd.DataFrame(model5.statistic_rules)
df_model5 = pd.DataFrame(model5.antecedent_description, columns=['Subgrupros'])
result_model5 = pd.concat([df_model5, model_estat5], axis=1)
result_model5['WRAcc'] = (result_model5['usage']/result_model5['usage_default'])*((result_model5['mean'])-(result_model5['RSS_default_pattern']/result_model5['usage_default']))

#### Modelo 6

In [83]:
model_estat6 = pd.DataFrame(model6.statistic_rules)
df_model6 = pd.DataFrame(model6.antecedent_description, columns=['Subgrupros'])
result_model6 = pd.concat([df_model6, model_estat6], axis=1)
result_model6['WRAcc'] = (result_model6['usage']/result_model6['usage_default'])*((result_model6['mean'])-(result_model6['RSS_default_pattern']/result_model6['usage_default']))

### Salvando os experimentos

In [105]:
experimentos = 'experimentos_sd.xlsx'

with pd.ExcelWriter(experimentos) as writer:
    result_model.to_excel(writer, sheet_name='SSD_modelo_1', index=False)
    result_model2.to_excel(writer, sheet_name='SSD_modelo_2', index=False)
    result_model3.to_excel(writer, sheet_name='SSD_modelo_3', index=False)
    result_model4.to_excel(writer, sheet_name='SSD_modelo_4', index=False)
    result_model5.to_excel(writer, sheet_name='SSD_modelo_5', index=False)
    result_model6.to_excel(writer, sheet_name='SSD_modelo_6', index=False)
    result_beam_search.to_excel(writer, sheet_name='Beam_Search', index=False)
    result_dfs.to_excel(writer, sheet_name='DFS', index=False)